# Module 7: Finding Natural Groups

**Goal:** Segment customers into meaningful groups without labels

**Time:** ~20 minutes

**What you'll do:**
1. Apply K-Means clustering to customer data
2. Interpret what each cluster represents
3. Use clusters for business insights
4. Detect anomalies with Isolation Forest

---

## Setup

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/streamcart_customers.csv')

print(f"Loaded {len(df):,} customers")

---

## Part 1: Prepare Features for Clustering

Clustering requires **scaling** - features must be on similar ranges.

In [ ]:
# Select features that describe customer behavior
cluster_features = [
    'tenure_months',
    'logins_last_30d',
    'orders_total',
    'avg_order_value',
    'support_tickets_total'
]

X = df[cluster_features].fillna(0)

# Scale features (CRITICAL for clustering)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Before scaling:")
print(X.describe().round(1))

print("\nAfter scaling (all ~0 mean, ~1 std):")
print(pd.DataFrame(X_scaled, columns=cluster_features).describe().round(2))

---

## Part 2: K-Means Clustering

Group customers into K clusters based on similarity.

In [ ]:
# TODO: Train K-Means with 4 clusters

kmeans = None  # Replace with your code

# Uncomment when ready:
# kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
# kmeans.fit(X_scaled)

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

assert kmeans is not None, "Create the KMeans model first!"
assert hasattr(kmeans, 'labels_'), "Model not fitted—did you call .fit()?"

print(f"✓ K-Means fitted with {kmeans.n_clusters} clusters")
print(f"\nCluster sizes:")
for i in range(kmeans.n_clusters):
    count = (kmeans.labels_ == i).sum()
    print(f"  Cluster {i}: {count:,} ({count/len(df):.1%})")

---

## Part 3: Interpret the Clusters

What KIND of customer is in each cluster?

In [ ]:
# Add cluster labels to dataframe
df['cluster'] = kmeans.labels_

# Profile each cluster
cluster_profiles = df.groupby('cluster')[cluster_features + ['churn_30d']].mean()

print("=== Cluster Profiles ===")
print(cluster_profiles.round(1))

In [ ]:
# Visualize cluster profiles
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(cluster_features + ['churn_30d']):
    cluster_means = df.groupby('cluster')[feature].mean()
    axes[i].bar(cluster_means.index, cluster_means.values)
    axes[i].set_title(feature)
    axes[i].set_xlabel('Cluster')

plt.tight_layout()
plt.show()

In [ ]:
# TODO: Name the clusters based on their profiles
#
# Look at the profiles above. What pattern does each cluster show?

cluster_names = {
    0: "???",  # What kind of customer?
    1: "???",
    2: "???",
    3: "???"
}

# Example naming:
# - High tenure + high orders = "Loyal Enthusiasts"
# - Low tenure + high support = "Struggling Newbies"
# - High AOV + low frequency = "Occasional Big Spenders"

---

## Part 4: Choosing K (Number of Clusters)

How do you know 4 is the right number?

In [ ]:
# Elbow method: plot inertia vs K
inertias = []
k_range = range(2, 10)

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_scaled)
    inertias.append(km.inertia_)

plt.figure(figsize=(10, 5))
plt.plot(k_range, inertias, 'bo-')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia (lower = tighter clusters)')
plt.title('Elbow Method for Choosing K')
plt.grid(True, alpha=0.3)
plt.show()

print("💡 Look for the 'elbow' - where adding more clusters stops helping much")

---

## Part 5: Anomaly Detection

Find customers that don't fit ANY normal pattern.

In [ ]:
# Isolation Forest for anomaly detection
iso_forest = IsolationForest(contamination=0.05, random_state=42)  # Expect ~5% anomalies
anomaly_labels = iso_forest.fit_predict(X_scaled)

# -1 = anomaly, 1 = normal
df['is_anomaly'] = (anomaly_labels == -1).astype(int)

print(f"=== Anomaly Detection ===")
print(f"Normal customers: {(df['is_anomaly'] == 0).sum():,}")
print(f"Anomalies: {df['is_anomaly'].sum():,} ({df['is_anomaly'].mean():.1%})")

In [ ]:
# What do anomalies look like?
print("=== Anomaly Profiles ===")
print("\nNormal customers:")
print(df[df['is_anomaly'] == 0][cluster_features].describe().round(1))

print("\nAnomalies:")
print(df[df['is_anomaly'] == 1][cluster_features].describe().round(1))

In [ ]:
# Do anomalies churn more?
normal_churn = df[df['is_anomaly'] == 0]['churn_30d'].mean()
anomaly_churn = df[df['is_anomaly'] == 1]['churn_30d'].mean()

print(f"\nChurn rate:")
print(f"  Normal: {normal_churn:.1%}")
print(f"  Anomaly: {anomaly_churn:.1%}")
print(f"  Lift: {anomaly_churn/normal_churn:.1f}x")

---

## Part 6: Business Applications

In [ ]:
applications = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  USE CASE              │  HOW TO USE CLUSTERS                              │
├─────────────────────────────────────────────────────────────────────────────┤
│  Marketing             │  Different messaging per segment                  │
│                        │  "Loyal Enthusiasts" → referral program           │
│                        │  "At Risk" → retention offer                      │
├─────────────────────────────────────────────────────────────────────────────┤
│  Product               │  Identify underserved segments                    │
│                        │  What do high-value customers want?               │
├─────────────────────────────────────────────────────────────────────────────┤
│  Pricing               │  Price sensitivity by segment                     │
│                        │  Which segments are price-elastic?                │
├─────────────────────────────────────────────────────────────────────────────┤
│  Anomaly Detection     │  Flag suspicious accounts                         │
│                        │  Fraud, bot accounts, data entry errors           │
└─────────────────────────────────────────────────────────────────────────────┘
"""
print(applications)

---

## 📝 Final Exercise: Explain It

The marketing team asks: "Can you give us 4 customer segments to target differently?"

Write a 4-5 sentence summary of the clusters you found, with a suggested action for each.

In [ ]:
# Write your response:

marketing_response = """
YOUR RESPONSE HERE
"""

print(marketing_response)

---

## ✅ Module 7 Complete!

**What you learned:**
- How K-Means groups customers by similarity
- Why scaling is critical for clustering
- How to interpret and name clusters
- How Isolation Forest finds anomalies

In [ ]:
# Summary
print("=== Module 7 Summary ===")
print(f"\nClusters: {kmeans.n_clusters}")
for i in range(kmeans.n_clusters):
    count = (df['cluster'] == i).sum()
    churn = df[df['cluster'] == i]['churn_30d'].mean()
    print(f"  Cluster {i}: {count:,} customers, {churn:.1%} churn")

print(f"\nAnomalies detected: {df['is_anomaly'].sum():,}")

**Next:** [Module 8: Similarity and Embeddings →](./module_08_embeddings.ipynb)